In [173]:
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
%matplotlib inline
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

In [174]:
dfh = pd.read_csv("heart2.csv")
dfh.head(2)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.000,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.100,0,0,3,0


In [175]:
pd.set_option("display.float", "{:.3f}".format)
dfh.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,1025.000,1025.000,1025.000,1025.000,1025.000,1025.000,1025.000,1025.000,1025.000,1025.000,1025.000,1025.000,1025.000,1025.000
mean,54.434,0.696,0.942,131.612,246.000,0.149,0.530,149.114,0.337,1.072,1.385,0.754,2.324,0.513
std,9.072,0.460,1.030,17.517,51.593,0.357,0.528,23.006,0.473,1.175,0.618,1.031,0.621,0.500
min,29.000,0.000,0.000,94.000,126.000,0.000,0.000,71.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,48.000,0.000,0.000,120.000,211.000,0.000,0.000,132.000,0.000,0.000,1.000,0.000,2.000,0.000
50%,56.000,1.000,1.000,130.000,240.000,0.000,1.000,152.000,0.000,0.800,1.000,0.000,2.000,1.000
75%,61.000,1.000,2.000,140.000,275.000,0.000,1.000,166.000,1.000,1.800,2.000,1.000,3.000,1.000
max,77.000,1.000,3.000,200.000,564.000,1.000,2.000,202.000,1.000,6.200,2.000,4.000,3.000,1.000


In [176]:
dfh.target.value_counts()

1    526
0    499
Name: target, dtype: int64

In [177]:
cat_val = []
num_val = []
for column in dfh.columns:
    if len(dfh[column].unique()) <= 10:
        cat_val.append(column)
    else:
        num_val.append(column)

In [178]:
cat_val

['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal', 'target']

In [179]:
num_val

['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

In [180]:
preprocessor = ColumnTransformer(
    transformers=[
        ('sc', StandardScaler(), ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']),
        ('ohe', OneHotEncoder(), ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal'])])

In [181]:
X = dfh.drop('target', axis=1)
y = dfh.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [182]:
X

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,52,1,0,125,212,0,1,168,0,1.000,2,2,3
1,53,1,0,140,203,1,0,155,1,3.100,0,0,3
2,70,1,0,145,174,0,1,125,1,2.600,0,0,3
3,61,1,0,148,203,0,1,161,0,0.000,2,1,3
4,62,0,0,138,294,1,1,106,0,1.900,1,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.000,2,0,2
1021,60,1,0,125,258,0,0,141,1,2.800,1,1,3
1022,47,1,0,110,275,0,0,118,1,1.000,1,1,2
1023,50,0,0,110,254,0,0,159,0,0.000,2,0,2


In [183]:
preprocessor.fit_transform(X_train,X_test)

array([[ 0.51227152,  0.46363159, -0.48467262, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.40223597, -0.2188468 , -0.58820922, ...,  0.        ,
         0.        ,  1.        ],
       [-1.1382618 , -0.78757879, -0.04981887, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.36801292,  0.46363159,  1.13049845, ...,  0.        ,
         0.        ,  1.        ],
       [-1.24829735, -1.24256438, -0.69174583, ...,  0.        ,
         0.        ,  1.        ],
       [-0.25797736, -1.12881798, -0.29830673, ...,  0.        ,
         1.        ,  0.        ]])

In [184]:

def aff_score(clf, X_test, y_test):
    pred = clf.predict(X_test)
    clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
    print("Resultat:\n================================================")        
    print(f"Score de précision: {accuracy_score(y_test, pred) * 100:.3f}%")
    print("_______________________________________________")
    print(f"RAPPORT DE CLASSIFICATION:\n{clf_report}")
    print("_______________________________________________")
    print(f"Matrix de Confusion: \n {confusion_matrix(y_test, pred)}\n")

In [185]:
rfc = RandomForestClassifier(n_estimators=1000, random_state=42)
rfc.fit(X_train, y_train)

aff_score(rfc, X_test, y_test)

Resultat:
Score de précision: 98.052%
_______________________________________________
RAPPORT DE CLASSIFICATION:
                0       1  accuracy  macro avg  weighted avg
precision   0.964   1.000     0.981      0.982         0.981
recall      1.000   0.960     0.981      0.980         0.981
f1-score    0.981   0.979     0.981      0.980         0.980
support   159.000 149.000     0.981    308.000       308.000
_______________________________________________
Matrix de Confusion: 
 [[159   0]
 [  6 143]]



In [186]:
test_score = accuracy_score(y_test, rfc.predict(X_test)) * 100

res = pd.DataFrame(data=[["Random Forest Classifier",  test_score]], columns=['Model', 'Test d''exactitude %'])

res

,Model,Test dexactitude %
0,Random Forest Classifier,98.052


In [187]:
xgb = XGBClassifier(use_label_encoder=False)
xgb.fit(X_train, y_train)

aff_score(xgb, X_test, y_test)

Resultat:
Score de précision: 99.026%
_______________________________________________
RAPPORT DE CLASSIFICATION:
                0       1  accuracy  macro avg  weighted avg
precision   0.981   1.000     0.990      0.991         0.990
recall      1.000   0.980     0.990      0.990         0.990
f1-score    0.991   0.990     0.990      0.990         0.990
support   159.000 149.000     0.990    308.000       308.000
_______________________________________________
Matrix de Confusion: 
 [[159   0]
 [  3 146]]



In [188]:
test_score = accuracy_score(y_test, xgb.predict(X_test)) * 100

res1 = pd.DataFrame(data=[["XGBoost Classifier",  test_score]], columns=['Model', 'Test d''exactitude %'])
res = res.append(res1, ignore_index=True)
res

,Model,Test dexactitude %
0,Random Forest Classifier,98.052
1,XGBoost Classifier,99.026


In [189]:
n_estimators = [500, 900, 1100, 1500]
max_features = ['auto', 'sqrt']

max_depth = [2, 3, 5, 10, 15, None]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]

params = {
    'n_estimators': n_estimators, 
    'max_features': max_features,
    'max_depth': max_depth, 
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf
              }

rfc = RandomForestClassifier(random_state=42)
rfcv = GridSearchCV(rfc, params, scoring="accuracy", cv=3, verbose=1, n_jobs=-1)
rfcv.fit(X_train, y_train)
best_params = rfcv.best_params_
print(f"Meilleurs parametres: {best_params}")

rfc = RandomForestClassifier(**best_params)
rfc.fit(X_train, y_train)

aff_score(rfc, X_test, y_test)

Fitting 3 folds for each of 432 candidates, totalling 1296 fits


/Users/Espoir/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/Espoir/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/Espoir/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this paramet

Meilleurs parametres: {'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 1100}


/Users/Espoir/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Resultat:
Score de précision: 98.052%
_______________________________________________
RAPPORT DE CLASSIFICATION:
                0       1  accuracy  macro avg  weighted avg
precision   0.964   1.000     0.981      0.982         0.981
recall      1.000   0.960     0.981      0.980         0.981
f1-score    0.981   0.979     0.981      0.980         0.980
support   159.000 149.000     0.981    308.000       308.000
_______________________________________________
Matrix de Confusion: 
 [[159   0]
 [  6 143]]



In [190]:
test_score = accuracy_score(y_test, rfc.predict(X_test)) * 100

updres = pd.DataFrame(data=[["Tuned Random Forest Classifier", test_score]], columns=['Model', 'Testing Accuracy %'])
updres

,Model,Testing Accuracy %
0,Tuned Random Forest Classifier,98.052


In [191]:
param = dict(
    n_estimators=stats.randint(10, 1000),
    max_depth=stats.randint(1, 10),
    learning_rate=stats.uniform(0, 1)
)

xgb = XGBClassifier(use_label_encoder=False)
xgbcv = RandomizedSearchCV(xgb, param, cv=5, n_iter=50, scoring='accuracy', n_jobs=-1, verbose=1)
xgbcv.fit(X_train, y_train)
best_params = xgbcv.best_params_
print(f"Meilleurs parametres: {best_params}")

xgb = XGBClassifier(**best_params)
xgb.fit(X_train, y_train)


aff_score(xgb, X_test, y_test)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Meilleurs parametres: {'learning_rate': 0.6429844968371161, 'max_depth': 9, 'n_estimators': 101}
Resultat:
Score de précision: 99.026%
_______________________________________________
RAPPORT DE CLASSIFICATION:
                0       1  accuracy  macro avg  weighted avg
precision   0.981   1.000     0.990      0.991         0.990
recall      1.000   0.980     0.990      0.990         0.990
f1-score    0.991   0.990     0.990      0.990         0.990
support   159.000 149.000     0.990    308.000       308.000
_______________________________________________
Matrix de Confusion: 
 [[159   0]
 [  3 146]]



In [192]:
test_score = accuracy_score(y_test, xgb.predict(X_test)) * 100

res1 = pd.DataFrame(data=[["Tuned XGBoost Classifier", test_score]], columns=['Model','Testing Accuracy %'])
pd.concat([updres, res1],ignore_index=True)

,Model,Testing Accuracy %
0,Tuned Random Forest Classifier,98.052
1,Tuned XGBoost Classifier,99.026


In [203]:
fullpipe = Pipeline(steps =[
    
    ('preprocessor', preprocessor),
   
    ('model', XGBClassifier(learning_rate= 0.64298449683711615, max_depth= 9, n_estimators= 101))
])

In [204]:
model = XGBClassifier(learning_rate= 0.64298449683711615, max_depth= 9, n_estimators= 101)
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.6429844968371161, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=9, max_leaves=0,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=101, n_jobs=0, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, ...)

In [205]:
fullpipe

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('sc', StandardScaler(),
                                                  ['age', 'trestbps', 'chol',
                                                   'thalach', 'oldpeak']),
                                                 ('ohe', OneHotEncoder(),
                                                  ['sex', 'cp', 'fbs',
                                                   'restecg', 'exang', 'slope',
                                                   'ca', 'thal'])])),
                ('model',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=Non...
                               importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.6429844968371161, max_bin=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=9, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=101,
                               n_jobs=None, num_parallel_tree=None,
                               predictor=None, random_state=None,
                               reg_alpha=None, reg_lambda=None, ...))])

In [206]:
fullpipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('sc', StandardScaler(),
                                                  ['age', 'trestbps', 'chol',
                                                   'thalach', 'oldpeak']),
                                                 ('ohe', OneHotEncoder(),
                                                  ['sex', 'cp', 'fbs',
                                                   'restecg', 'exang', 'slope',
                                                   'ca', 'thal'])])),
                ('model',
                 XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, earl...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.6429844968371161, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=9, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=101, n_jobs=0, num_parallel_tree=1,
                               predictor='auto', random_state=0, reg_alpha=0,
                               reg_lambda=1, ...))])

In [207]:
# Pour maximiser la reproductibilité, nous aimerions utiliser ce pipeline à plusieurs reprises pour nos nouvelles données entrantes. 
# Sauvegardons le pipeline en utilisant le paquet 'joblib' pour le sauvegarder dans un fichier pickle.

joblib.dump(fullpipe, 'health_pipeline1.pkl')

# Maintenant nous pouvons appeler ce pipeline, qui inclut toutes sortes de prétraitements de données dont nous avons besoin.
loaded_pipeline = joblib.load('health_pipeline1.pkl')


In [208]:
dfh1 = pd.read_csv("heart2.csv")
dfh1 = dfh1.drop('target', axis=1)
dfh1.to_csv('test.csv', index=False)
dfh1.head(2)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,52,1,0,125,212,0,1,168,0,1.000,2,2,3
1,53,1,0,140,203,1,0,155,1,3.100,0,0,3


In [213]:
sample1 = [[34,0,1,118,210,0,1,192,0,0.7,2,0,2]]
sample2 = [[58,0,0,100,248,0,0,122,0,1.0,1,0,2]]

sample = pd.DataFrame(sample1, columns=['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal'])

In [214]:

if loaded_pipeline.predict(sample) == 0:
    print('PATIENT EN ETAT NORMAL')
else:
    print('ATTENTION!!! PATIENT EN ETAT CRITIQUE')

ATTENTION!!! PATIENT EN ETAT CRITIQUE
